# TECNMS-2401

# Table of Contents
<div class="lev1 toc-item"><a href="#Jupyter-Notebooks" data-toc-modified-id="Jupyter-Notebooks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Jupyter Notebooks</a></div>
<div class="lev1 toc-item"><a href="#Operational-Data-via-CLI-Automation" data-toc-modified-id="Operational-Data-via-CLI-Automation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Operational Data via CLI Automation</a></div>
<div class="lev1 toc-item"><a href="#NETCONF/YANG" data-toc-modified-id="NETCONF/YANG-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NETCONF/YANG</a></div>
<div class="lev2 toc-item"><a href="#NETCONF-Routing-Table" data-toc-modified-id="NETCONF-Routing-Table-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>NETCONF Routing Table</a></div>

<div class="lev2 toc-item"><a href="#YANG-Models" data-toc-modified-id="YANG-Models-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>YANG Models</a></div>

<div class="lev2 toc-item"><a href="#NETCONF-Operational-Data" data-toc-modified-id="NETCONF-Operational-Data-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>NETCONF Operational Data</a></div>

<div class="lev2 toc-item"><a href="#NETCONF-Configuration-Data" data-toc-modified-id="NETCONF-Configuration-Data-33"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>NETCONF Configuration Data</a></div>

<div class="lev1 toc-item"><a href="#RESTCONF" data-toc-modified-id="RESTCONF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>RESTCONF</a></div>
<div class="lev1 toc-item"><a href="#YDK" data-toc-modified-id="YDK-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>YDK</a></div>

<div class="lev1 toc-item"><a href="#Telemetry" data-toc-modified-id="Telemetry-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Telemetry</a></div>

## Jupyter Notebooks
The *Jupyter Notebooks* in this workbench class are interactive learning environments. We are using them to learn about network device programability in combination with virtual Cisco network devices.

Notebooks are **interactive**, the code and text in each notebook can be changed on the fly and the result of those changes can be seen immediately within the notebook itself.

Notebooks are organized in *cells*, each displayed *rectangle* or *cell* is basically either Markdown text (like this cell) or Python code. Python code can be *executed* by either clicking the  <span class="inline">![](images/forwardicon.png)</span> in the 'Cell' menu in the menu bar or the key combination 'Shift-Enter'.

<div class="my-notify-warning">

<p>**IMPORTANT for Execution. PLEASE READ**. At its simplest, this is a playbook and execution is top-down. Code cells will look something like `In [#]:` (with '#' being a number). That's where you want to single-click. To execute the code, simply hit Shift+Enter.</p>

<p>The code should execute, and move onto the next cell. The number in `In [#]:` will change and if there is some return value (as opposed to an explicit `print()` statement) it will appear in an `Out [#]:` cell. The numbers will change after each cell execution.</p>

<p>It's easy to "execute" an entire notebook, task-by-task, by continuing to execute Shift+Return. Also, if you ever see `In [*]:` in your code cells (note the asterisk '\*'), it just means things are executing, and you are probably waiting on data to come back.</p>

<p>So depending on what you're doing, some patience might be required.</p>

</div>


## Environment
The workbench environment is running entirely inside of the provided MacBook Pros. We use [Vagrant](https://www.vagrantup.com/) to run two [CSR 1000V virtual routers](http://www.cisco.com/c/en/us/products/routers/cloud-services-router-1000v-series/index.html) running IOS XE 16.3.2 and we use [Docker](https://www.docker.com/) to provide the [Jupyter](http://jupyter.org/) interactive notebooks (like the document you are currently looking at).

The Python environment in the notebooks communicate with the routers via a host-only *internal* network on the MacBook Pro. The MacBook has its own IP address on this host-only network: `172.20.20.1`. The container can access the routers on their own IP addresses, `172.20.20.10` and `172.20.20.20` respectively. The following diagram shows a high level overview of the environment:

![Topology](images/topology.png)

We can access the CLI of the CSR1000v using a terminal (`Terminal.app`) and, while in the Vagrant directory, typing `vagrant ssh rtr1` or `vagrant ssh rtr2`. Because a public key has been installed into the router(s), we get a privileged shell / CLI without being prompted for a password.

While on the router, you can check the 'wsma' configuration section and also the remaining configuration which enables NETCONF and RESTCONF. 

Apart from a few static routes and an OSPF process there is not a lot else configured on the box.

# Operational Data via CLI Automation
Here we will set variables that we will use thoughout the session

In [2]:
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

Programmability has been around for a long time over the CLI API.  We all use it for FDN, but there are capabilities added over the years to communicate with devices via Python over ssh leveraging some form of python expect.  In this case we will use a tool called Netmiko.  However the format of the data is meant for human consumption, not a machine interface

In [3]:
import jtextfsm as textfsm
from netmiko import ConnectHandler
import pprint

def get_show_inventory(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show inventory", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result


If we are lucky there is some clear format of the response that we can leverage regular expression filters to turn the response into programming data types

In [4]:
    raw_text_data = get_show_inventory(HOST, USER, PASS, PASS)
    print ('This is what the text looks like\n' + raw_text_data)
    # Run the text through the FSM. 
    # The argument 'template' is a file handle and 'raw_text_data' is a 
    # string with the content from the show_inventory.txt file
    template = open("/home/docker/cli/show_inventory_multiple.textfsm")
    re_table = textfsm.TextFSM(template)
    fsm_results = re_table.ParseText(raw_text_data)
    # build keys for inventory sub-items 
    keys = [s for s in re_table.header]
    #build inventory list of dictionaries
    inventory = []
    for row in fsm_results:
      entry = {}
      for i, item in enumerate(row):
        entry.update({keys[i]:item})
      inventory.append(entry)
    pp = pprint.PrettyPrinter(indent=2)

This is what the text looks like
rtr1#
NAME: "Chassis", DESCR: "Cisco CSR1000V Chassis"
PID: CSR1000V          , VID: V00  , SN: 9K7FMZNX8PK

NAME: "module R0", DESCR: "Cisco CSR1000V Route Processor"
PID: CSR1000V          , VID: V00  , SN: JAB1303001C

NAME: "module F0", DESCR: "Cisco CSR1000V Embedded Services Processor"
PID: CSR1000V          , VID:      , SN:            




Here we can see what the template looks like

In [5]:
    print(template.read())

Value Filldown hostname (\S+)
Value name (.+)
Value description (.*)
Value productid (\S*)
Value vid (\S*)
Value Required serialnumber (\S+)

Start
  ^${hostname}[>#].*
  ^NAME: "${name}", DESCR: "${description}"
  ^PID: ${productid}.*VID: ${vid}.*SN: ${serialnumber} -> Record


Now that we have srtuctured data, lets do something interesting with it.  We can choose how we display it or to even search for a value of interest.

In [6]:
    print('Results translated through regular expression template organized as a dictonary')
    pp.pprint(inventory) # Print full inventory as dictionary
    print([key for key, value in inventory[0].items()])  # print key values as list
    for item in inventory:   # print inventory elements a table 
        print([item[key] for key in item])
    

Results translated through regular expression template organized as a dictonary
[ { u'description': u'Cisco CSR1000V Chassis',
    u'hostname': u'rtr1',
    u'name': u'Chassis',
    u'productid': u'CSR1000V',
    u'serialnumber': u'9K7FMZNX8PK',
    u'vid': u'V00'},
  { u'description': u'Cisco CSR1000V Route Processor',
    u'hostname': u'rtr1',
    u'name': u'module R0',
    u'productid': u'CSR1000V',
    u'serialnumber': u'JAB1303001C',
    u'vid': u'V00'}]
[u'serialnumber', u'name', u'vid', u'description', u'hostname', u'productid']
[u'9K7FMZNX8PK', u'Chassis', u'V00', u'Cisco CSR1000V Chassis', u'rtr1', u'CSR1000V']
[u'JAB1303001C', u'module R0', u'V00', u'Cisco CSR1000V Route Processor', u'rtr1', u'CSR1000V']


In [7]:
    sought_product = 'C9300-24P-A'
    resultlist = [d for d in inventory if d.get('productid', '') == sought_product]
    print('\nFound Devices with specific productid ' + sought_product)
    pp.pprint(resultlist) #print list of devices with sought productid
    #print ([(item['hostname'], item['productid']) for item in resultlist])  #print found items as a tuple


Found Devices with specific productid C9300-24P-A
[]


However, we are not always so lucky to have data that we can easily parse.

In [8]:
def get_show_ip_route(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show ip route", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result

In [9]:
raw_text_data = get_show_ip_route(HOST, USER, PASS, PASS)
print ('This is what the text looks like\n' + raw_text_data)

This is what the text looks like
rtr1#
Codes: L - local, C - connected, S - static, R - RIP, M - mobile, B - BGP
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area 
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2
       i - IS-IS, su - IS-IS summary, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, * - candidate default, U - per-user static route
       o - ODR, P - periodic downloaded static route, H - NHRP, l - LISP
       a - application route
       + - replicated route, % - next hop override, p - overrides from PfR

Gateway of last resort is 10.0.2.2 to network 0.0.0.0

S*    0.0.0.0/0 [254/0] via 10.0.2.2
      10.0.0.0/8 is variably subnetted, 2 subnets, 2 masks
C        10.0.2.0/24 is directly connected, GigabitEthernet1
L        10.0.2.15/32 is directly connected, GigabitEthernet1
      172.16.0.0/16 is variably subnetted, 2 subnets, 2 masks
C        172.16.

# NETCONF/YANG
So let's take a look at an aternative method of interacting with devices.  Let's consider the machine's needs and leverage a model built for it.  That brings us to NETCONF.  Let's try to grab the routing table via programmatic API

![](images/netconf_stack.png)


## NETCONF Routing Table

In [10]:
import ncclient.manager
import xml.dom.minidom
from ncclient.operations import TimeoutExpiredError
import xmltodict
import pprint
nckwargs = dict(
    host=HOST,
    port=PORT,
    hostkey_verify=False,
    username=USER,
    password=PASS,
    device_params={'name':"csr"}
)
m = ncclient.manager.connect(**nckwargs)  

In [11]:
route_filter ='''    
    <filter>
      <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
        <routing-instance>
          <name>default</name>
          <ribs>
            <rib>
              <name>ipv4-default</name>
            </rib>
          </ribs>
        </routing-instance>
      </routing-state>
    </filter>'''
try:
    print ('Here we are printing the RIB as XML\n')
    c = m.get(route_filter)
    xmlDom = xml.dom.minidom.parseString(str(c))
    print (xmlDom.toprettyxml( indent = " " ))
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("ERORR severity={}, tag={}".format(e.severity, e.tag))

Here we are printing the RIB as XML

<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:a674c72b-529a-4e81-8ea5-869150ca952c" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
   <routing-instance>
    <name xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">default</name>
    <ribs>
     <rib>
      <name xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">ipv4-default</name>
      <address-family>ipv4</address-family>
      <default-rib>false</default-rib>
      <routes>
       <route>
        <destination-prefix>0.0.0.0</destination-prefix>
        <route-preference>254</route-preference>
        <metric>254</metric>
        <next-hop>
         <outgoing-interface/>
         <next-hop-address>10.0.2.2</next-hop-address>
        </next-hop>
        <source-protocol>static</source-protocol>
        <active/>
       </route>
       <route>
        <destination

Now that is a lot of information to look through, maybe you are only interested in a few pieces of information per route.  We can apply an XPATH filter, which is to say we will parse the response programmatically and return the interesting data in a condensed format

In [12]:
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)
route_table=[]
routes = c.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    prot = route.xpath('./rt:source-protocol/text()', namespaces=nsmap)
    if len(hop) > 0:
        entry = {'hop':hop[0]}
    if len(intfc) > 0:
        entry = {'intfc':intfc[0]}  
    entry.update({'dest':dest[0], 'prot':prot[0]}) 
    route_table.append(entry)
print ('\nOr we can just grab the information that is interesting\n')
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(route_table)  


Or we can just grab the information that is interesting

[ { 'dest': '0.0.0.0', 'hop': '10.0.2.2', 'prot': 'static'},
  { 'dest': '10.0.2.0', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '10.0.2.15', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '172.16.100.0', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.16.100.10', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.20.20.0', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '172.20.20.10', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '192.168.0.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.1.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.2.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.4.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.10.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.11.0', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': 

That seems easier to work with, but what are the items we can request?  In this case we can look at the Model.

## YANG Models
The best way to describe a YANG model is to take a look at one.  We can leverage NETCONF protocol to learn the capabilities of a device to include the det of Modles and even download the model schema itself and take a look.  Notice there are 2 types of models, open and native.

![  ](images/model_stack.png)


In [16]:
from subprocess import Popen, PIPE, STDOUT
#SCHEMA_TO_GET = 'Cisco-IOS-XE-memory-oper'
#SCHEMA_TO_GET = 'Cisco-IOS-XE-process-cpu-oper'
#SCHEMA_TO_GET = 'Cisco-IOS-XE-bgp'
SCHEMA_TO_GET = 'openconfig-interfaces'
#SCHEMA_TO_GET = 'openconfig-acl'
#SCHEMA_TO_GET = 'openconfig-rib-bgp'
SCHEMA_TO_GET = 'openconfig-vlan'
#SCHEMA_TO_GET = 'ietf-interfaces'
SCHEMA_TO_GET = 'Cisco-IOS-XE-platform-oper'
print(' We are connecting to the device and downloading the YANG Module')
print("Now we will do a 'pyang -f tree of %s.yang'" % SCHEMA_TO_GET)
print('This structure is what data we can get (ro/rw) or set (rw)')
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]
print stdout_data

 We are connecting to the device and downloading the YANG Module
Now we will do a 'pyang -f tree of Cisco-IOS-XE-platform-oper.yang'
This structure is what data we can get (ro/rw) or set (rw)
module: Cisco-IOS-XE-platform-oper
   +--ro components
      +--ro component* [cname]
         +--ro cname                     string
         +--ro state
         |  +--ro type?          platform-ios-xe-oper:platform-comp-type
         |  +--ro id?            string
         |  +--ro description?   string
         |  +--ro mfg-name?      string
         |  +--ro version?       string
         |  +--ro serial-no?     string
         |  +--ro part-no?       string
         |  +--ro temp
         |     +--ro temp-instant?   decimal64
         |     +--ro temp-avg?       decimal64
         |     +--ro temp-max?       decimal64
         |     +--ro temp-min?       decimal64
         +--ro platform-properties
         |  +--ro platform-property* [name]
         |     +--ro name            string
      

### Now that we know whats available, we can grab operational data for the device to give us some perspective of its state.

## NETCONF Operational Data

In [ ]:
cpu_util = '''<filter>
      <cpu-usage xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-process-cpu-oper">
        <cpu-utilization>
          <five-seconds/>
        </cpu-utilization>
      </cpu-usage>
</filter>'''

memory = '''<filter><memory-statistics xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-memory-oper">
        <memory-statistic>
          <total-memory/>
          <used-memory/>
        </memory-statistic>
      </memory-statistics></filter>'''

print ('Using NETCONF, you can present data as XML without any conversion')

response = str( m.get(cpu_util))

xmlDom = xml.dom.minidom.parseString(response)
print xmlDom.toprettyxml( indent = " " )

In [ ]:
cpu_util = '''
    <filter>
      <cpu-usage xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-process-cpu-oper">
        <cpu-utilization>
          <five-seconds/>
          <five-seconds-intr/>
          <one-minute/>
          <five-minutes/>
        </cpu-utilization>
      </cpu-usage>
    </filter>'''

cpu_process = '''
    <filter>
      <cpu-usage xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-process-cpu-oper">
        <cpu-utilization>
          <cpu-usage-processes>
            <cpu-usage-process/>
          </cpu-usage-processes>
        </cpu-utilization>
      </cpu-usage>
    </filter>'''
print ('Using NETCONF, you can present data as XML without any conversion')
cpu_util = str( m.get( filter=cpu_util ) )
xmlDom = xml.dom.minidom.parseString(cpu_util)
print xmlDom.toprettyxml( indent = " " )

In [ ]:
import xmltodict
print ('Or you can convert to Dictionary')
cpu_dict = xmltodict.parse( cpu_util )['rpc-reply']['data']
print [(i,v) for i,v in cpu_dict['cpu-usage']['cpu-utilization'].iteritems()]

With structured data you can select data thats interesting

In [ ]:
print ('\nWith structured data you can filter or sort \nHere we are grabbing the processes with higher run-time')
cpu_dict = xmltodict.parse( str( m.get( filter=cpu_process ) ) ) #['rpc-reply']['data']

#cpu_dict = xmltodict.parse(str(xml_result))['rpc-reply']['data']
#for process in cpu_dict['cpu-usage']['cpu-utilization']['cpu-usage-processes']['cpu-usage-process']:
#    process_name = process['name']
#    process_one_minute_utilization = float(process['one-minute'])
#    if process_one_minute_utilization > 0:
#        print("Process name: {}, one minute CPU utilization: {}%".format(process_name, process_one_minute_utilization))

cpu_dict = cpu_dict['rpc-reply']['data']
nonzero_process = [t for t in cpu_dict['cpu-usage']['cpu-utilization']['cpu-usage-processes']['cpu-usage-process'] if int(t['total-run-time']) > 100]
for i in nonzero_process: print (i['name'],i['pid'],i['total-run-time'])

## NETCONF Configuration Data

In [ ]:
CONF_FILTER ='''<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>'''
m = ncclient.manager.connect(**nckwargs)
print('\nHere is the XML data for Interfaces Configuration')
interfaces = m.get_config('running',CONF_FILTER)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))

# RESTCONF

RESTCONF is relative of NETCONF, born from YANG.  It provides access to the same data, but over a REST-like interface.  This extends access to YANG data to just about anything.  It is not a replacement to NETCONF, but instead an extension of a subset of the capabilities to REST capable systems.

![](images/restconf_stack.jpg)

In [ ]:
import requests
import json

PORT = 443
PROT = 'https'

API_JSON = 'application/yang-data+json'
DATA_JSON = 'application/yang-data+json'

def url(method=''):
    '''return the proper URL given the global vars and the
    method parameter.
    '''
    return "{prot}://{host}:{port}/restconf/{method}".format(prot=PROT, host=HOST, method=method, port=PORT)

def pp(data):
    '''pretty print the Python object
    '''
    if isinstance(data, requests.models.Response):
        if not data.ok:
            print('{}: {}'.format(data.status_code, data.text))
            return None
        obj = data.json()
    else:
        obj = data
    print(json.dumps(obj, indent=2))
    return obj

# Setup a global session with proper username and password
session = requests.Session()
session.auth = (USER, PASS)
# we also set the default dictionary with our 'Accept' header
session.headers = {'accept': DATA_JSON, 'content-type': DATA_JSON}

# are we using HTTPS?
if 'https' in PROT:
    # dont' verify certificates
    session.verify = False
    # disable warnings in case we use HTTPS
    requests.packages.urllib3.disable_warnings()

In [ ]:
o = pp(session.get(url(), headers=dict(accept=API_JSON)))

In [ ]:
o = pp(session.get(url('data/restconf-state')))

In [ ]:
o = pp(session.get(url('data/openconfig-interfaces:interfaces/interface=GigabitEthernet1')))

# YDK
Now that we are all Python experts, let's take a different perspective.  Models are similar to Programming languages.  Containers are to Classes, Lists are to Lists, Leafs are to Objects or types.

In a community effort we have "compiled" the YANG model in to Python code.  By importing a few libraries we can affect change on networking elements simply by manipulating code.  All you need to do to use is "pip install ydk" to install.

![](images/YDK.png)

In [ ]:
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider
from ydk.models.ietf import ietf_interfaces as intf
from ydk.models.ietf import iana_if_type as iftype

PORT = 830

# create NETCONF provider
provider = NetconfServiceProvider(address=HOST,
                                  port=PORT,
                                  username=USER,
                                  password=PASS)
# create CRUD service
crud = CRUDService()

# query object
q_i = intf.Interfaces()

# get stuff
intfs = crud.read(provider, q_i)

In [ ]:
# print interface names and types
for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

In [ ]:
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
int_info=[(i.name, i.statistics.out_pkts, int(i.speed)/1000000, i.oper_status) for i in intfs.interface]
for thing in int_info: print(str(thing))

## Create Software Loopback

In [ ]:
# new interface
new_loopback = intf.Interfaces.Interface()

# create a new loopback
new_loopback.name = 'Loopback666'
new_loopback.type = iftype.SoftwareloopbackIdentity()
new_loopback.description = 'Created by DevNet2449'
res = crud.create(provider, new_loopback)

In [ ]:
# get stuff
q_i = intf.Interfaces()
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

## Delete Software Loopback

In [ ]:
# create CRUD service
crud = CRUDService()

# interface to delete
to_del = intf.Interfaces.Interface()

# create a new loopback
to_del.name = 'Loopback666'
res = crud.delete(provider, to_del)

In [ ]:
# get stuff
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))
provider.close()

# Telemetry

Here is a new option to gather data.  Why use SNMP, Syslog, Netflow, etc when you can gather Model based telemetry as simply as sending a request to the device?

![](images/telemetry.png)

In [17]:
CLIENT = '172.16.54.145'
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

In [18]:
import requests

headers = {'Content-Type': 'application/json',}
# Grab telemetry on CPU utilization
data = '{"xpath":"/process-cpu-ios-xe-oper:cpu-usage/cpu-utilization/five-seconds", "period": 1000, "incident_id": 1234}'
response = requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)
print response

<Response [200]>


In [19]:
# Grab telemetry on Free Memory
data = '{"xpath":"/memory-ios-xe-oper:memory-statistics/memory-statistic/free-memory","period": 1000, "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)

<Response [200]>

In [20]:
# Grab telemetry on Free Memory
data = '{"xpath":"/memory-ios-xe-oper:memory-statistics/memory-statistic/used-memory","period": 1000, "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)

<Response [200]>

In [21]:
# Grab telemetry on Input 
data = '{"xpath":"/if:interfaces-state/interface[name=&quot;GigabitEthernet1&quot;]/statistics/in-octets","period": 1000,  "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)

<Response [200]>

In [22]:
# Grab telemetry on Output
data = '{"xpath":"/if:interfaces-state/interface[name=&quot;GigabitEthernet1&quot;]/statistics/out-octets","period": 1000,  "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)

<Response [200]>

In [23]:
# Grab telemetry on Output
data = '{"xpath":"/environment-ios-xe-oper:environment-sensors/environment-sensor/current-reading","period": 1000,  "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)

<Response [200]>

In [24]:
# Grab telemetry on ACL Hit
data = '{"xpath":"/oc-acl:acl/acl-sets/acl-set[name=&quot;DEMO&quot;]acl-entries/acl-entry[sequence-id=&quot;10&quot;]/state/matched-packets","period": 1000,  "incident_id": 1234}'
requests.post('http://%s:8320/sendrpc'% (CLIENT), headers=headers, data=data)


<Response [200]>

Take a look at Your ELK Stack (that you built outside this Notebook.  You have...Useful Information!

Telemetry Stack
![](images/telemetry_stack.png)